In [1]:
import sys
sys.path.append('/home/mz1482/project/vt-bayesian-opt-bopt_debug/my_bayes_opt/')
import bo_new
import pandas as pd
import numpy as np
from data_analysis import get_heart_bounds, correlation_coef, graph_3d, get_index
from graph import narrow,corrplot3axes,trend,nearest,plot_exploration, graph_cc_distribution, cube, gp_plot, gp_plot2,predicted_visited, init_gp_plot
from BayesOptLib.bayes_opt.bayesian_optimization import BayesianOptimization
from RandomSampler import RandomSampler
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics.pairwise import euclidean_distances
matplotlib.use('Qt5Agg')  # or can use 'TkAgg', whatever you have/prefer

In [2]:
def black_box(x, y, z):
    """
    Represents a black box function to maximize for CC of two ECGs given an XYZ coordinate
    :param x:
    :param y: coordinates of the prediction
    :param z:
    :return: CC of that point and the target
    """
    sample_ecg = ecgs[get_index(np.array([x, y, z]),labels)]
    return abs(correlation_coef(target_ecg, sample_ecg))

In [3]:
ecgs = pd.read_csv("new_simu-data/Heart1/Heart1_SimuData_4000.csv", header=None).to_numpy()
labels = pd.read_csv("new_simu-data/Heart1/Coord1_4000.csv", header=None).to_numpy() / 1000
init = 5
steps = 10

In [4]:
bounds = get_heart_bounds(labels)
# Pick out a sample to use as a target
tidx = np.random.randint(0, labels.shape[0])
target, target_ecg = labels[tidx], ecgs[tidx]
# optimizer = optimize_point(labels,bounds)  
optimizer = bo_new.mybo(f=black_box,pbounds=bounds, real_set=labels)
gp,X,rs,predicted = optimizer.gpfit(init_points=init, n_iter=steps,  acq="ucb", kappa = 1)
# print(optimizer.predicted)
graph_cc_distribution(target_ecg,ecgs,labels)
init_gp_plot(init,gp,labels,X)
predicted_visited(init,target,target_ecg,labels,ecgs,X,predicted)

Initialization
-----------------------------------------------------------------
 Step |   Time |      Value |         x |         y |         z | 
    1 | 00m00s |    0.13487 |   -7.6837 |  -16.0327 |  -15.6516 | 
    2 | 00m00s |    0.19841 |   15.5183 | -101.6560 |    5.0517 | 
    3 | 00m00s |    1.00000 |   -6.1670 |  -88.3709 |  -36.6618 | 
    4 | 00m00s |    0.13284 |    0.1864 |  -98.4354 |   21.7536 | 
    5 | 00m00s |    0.11688 |    5.5928 |  -82.5250 |   18.5371 | 
Bayesian Optimization
-----------------------------------------------------------------
 Step |   Time |      Value |         x |         y |         z | 
    6 | 00m03s |    0.76416 |  -48.5647 |  -94.1265 |  -31.0406 | 
    7 | 00m03s |    0.96937 |   -7.3430 |  -99.4235 |  -38.5416 | 
    8 | 00m02s |    0.81032 |  -12.2912 |  -75.0930 |  -43.4956 | 
    9 | 00m03s |    0.55158 |   25.2813 |  -89.9660 |  -40.3020 | 
   10 | 00m02s |    0.94324 |  -22.5091 |  -98.9788 |  -29.3956 | 
   11 | 00m02s |    0.86000

In [5]:
predicted

array([[ -59.2155    , -112.304     ,  -47.3608    ],
       [ -11.63950582, -112.304     ,  -47.3608    ],
       [ -15.66761945,  -75.28601112,  -47.3608    ],
       [  24.44349289,  -87.40303563,  -47.3608    ],
       [ -18.65305138,  -92.09573485,  -22.46635105],
       [ -10.56327285,  -89.03536669,  -26.95167487],
       [ -15.73746912,  -93.00736387,  -41.7001247 ],
       [  -9.35980973,  -92.80107696,  -34.15104716],
       [ -59.2155    , -112.304     ,   -4.35315075],
       [  -4.4742547 ,  -91.21414241,  -42.23366481],
       [ -11.01552629,  -90.07735745,  -35.92449515]])